# Fine Tuning a gpt-35-turbo model

Source: [Azure OpenAI GPT 3.5 Turbo fine-tuning tutorial](https://learn.microsoft.com/azure/ai-services/openai/tutorials/fine-tune?tabs=python-new%2Ccommand-line?WT.mc_id=academic-105485-koreyst)

https://learn.microsoft.com/en-us/azure/ai-services/openai/tutorials/fine-tune?tabs=python-new%2Ccommand-line

## Setup

You'll need to create two files training_set.jsonl and validation_set.jsonl.

Create the files in the same directory that you're running the Jupyter Notebook, and copy the contents of the following code blocks to the corresponding files: 

Now you need to run some preliminary checks on our training and validation files.

In [2]:
import json

# Load the training set
with open('training_set.jsonl', 'r', encoding='utf-8') as f:
    training_dataset = [json.loads(line) for line in f]

# Training dataset stats
print("Number of examples in training set:", len(training_dataset))
print("First example in training set:")
for message in training_dataset[0]["messages"]:
    print(message)

# Load the validation set
with open('validation_set.jsonl', 'r', encoding='utf-8') as f:
    validation_dataset = [json.loads(line) for line in f]

# Validation dataset stats
print("\nNumber of examples in validation set:", len(validation_dataset))
print("First example in validation set:")
for message in validation_dataset[0]["messages"]:
    print(message)

Number of examples in training set: 10
First example in training set:
{'role': 'system', 'content': 'Clippy is a factual chatbot that is also sarcastic.'}
{'role': 'user', 'content': 'Who discovered Antarctica?'}
{'role': 'assistant', 'content': "Some chaps named Fabian Gottlieb von Bellingshausen and Mikhail Lazarev, as if they don't teach that in every school!"}

Number of examples in validation set: 10
First example in validation set:
{'role': 'system', 'content': 'Clippy is a factual chatbot that is also sarcastic.'}
{'role': 'user', 'content': "What's the capital of Australia?"}
{'role': 'assistant', 'content': "It's Canberra, not Sydney. Shocking, I know!"}


Now you can then run some additional code from OpenAI using the tiktoken library to validate the token counts. Individual examples need to remain under the gpt-35-turbo-0613 model's input token limit of 4096 tokens

In [3]:
%pip install tiktoken

     -------------------------------------- 798.7/798.7 kB 3.6 MB/s eta 0:00:00
     -------------------------------------- 269.5/269.5 kB 5.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import json
import tiktoken
import numpy as np
from collections import defaultdict

encoding = tiktoken.get_encoding("cl100k_base") # default encoding used by gpt-4, turbo, and text-embedding-ada-002 models

def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

files = ['training_set.jsonl', 'validation_set.jsonl']

for file in files:
    print(f"Processing file: {file}")
    with open(file, 'r', encoding='utf-8') as f:
        dataset = [json.loads(line) for line in f]

    total_tokens = []
    assistant_tokens = []

    for ex in dataset:
        messages = ex.get("messages", {})
        total_tokens.append(num_tokens_from_messages(messages))
        assistant_tokens.append(num_assistant_tokens_from_messages(messages))
    
    print_distribution(total_tokens, "total tokens")
    print_distribution(assistant_tokens, "assistant tokens")
    print('*' * 50)

Processing file: training_set.jsonl

#### Distribution of total tokens:
min / max: 47, 62
mean / median: 52.1, 50.5
p5 / p95: 47.9, 57.5

#### Distribution of assistant tokens:
min / max: 13, 30
mean / median: 17.6, 15.5
p5 / p95: 13.0, 21.9
**************************************************
Processing file: validation_set.jsonl

#### Distribution of total tokens:
min / max: 43, 65
mean / median: 51.4, 49.0
p5 / p95: 45.7, 56.9

#### Distribution of assistant tokens:
min / max: 8, 29
mean / median: 15.9, 13.5
p5 / p95: 11.6, 20.9
**************************************************


In [5]:
# Upload fine-tuning files

import os
from openai import AzureOpenAI

from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT_FINETUNE"), 
  api_key=os.getenv("AZURE_OPENAI_KEY_FINETUNE"),  
  api_version="2023-12-01-preview"  # This API version or later is required to access fine-tuning for turbo/babbage-002/davinci-002
)

training_file_name = 'training_set.jsonl'
validation_file_name = 'validation_set.jsonl'

# Upload the training and validation dataset files to Azure OpenAI with the SDK.

training_response = client.files.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
training_file_id = training_response.id

validation_response = client.files.create(
    file=open(validation_file_name, "rb"), purpose="fine-tune"
)
validation_file_id = validation_response.id

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-37f7bd78e5be46d99411270551df055e
Validation file ID: file-942d429695404cfca2c0aef334ccbbd1


## Begin fine-tuning

Now that the fine-tuning files have been successfully uploaded you can submit your fine-tuning training job:



In [12]:
response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model="gpt-35-turbo-0613", # Enter base model name. Note that in Azure OpenAI the model name contains dashes and cannot contain dot/period characters. 
)

job_id = response.id

# You can use the job ID to monitor the status of the fine-tuning job.
# The fine-tuning job will take some time to start and complete.

print("Job ID:", response.id)
print("Status:", response.id)
print(response.model_dump_json(indent=2))

Job ID: ftjob-4c94f760d27141c587b7ac90d74ac538
Status: ftjob-4c94f760d27141c587b7ac90d74ac538
{
  "id": "ftjob-4c94f760d27141c587b7ac90d74ac538",
  "created_at": 1711621725,
  "error": null,
  "fine_tuned_model": null,
  "finished_at": null,
  "hyperparameters": {
    "n_epochs": 2,
    "batch_size": 1,
    "learning_rate_multiplier": 1
  },
  "model": "gpt-35-turbo-0613",
  "object": "fine_tuning.job",
  "organization_id": null,
  "result_files": null,
  "status": "pending",
  "trained_tokens": null,
  "training_file": "file-199f51f296464e2183b8480182c13f57",
  "validation_file": "file-16775f66500f4123bfc5fef422080068",
  "updated_at": 1711621725
}


## Track Training Status
If you would like to poll the training job status until it's complete, you can run:

In [7]:
# Track training status

from IPython.display import clear_output
import time

start_time = time.time()

# Get the status of our fine-tuning job.
#job_id="ftjob-4c94f760d27141c587b7ac90d74ac538"
response = client.fine_tuning.jobs.retrieve(job_id)

status = response.status

# If the job isn't done yet, poll it every 10 seconds.
while status not in ["succeeded", "failed"]:
    time.sleep(10)
    
    response = client.fine_tuning.jobs.retrieve(job_id)
    print(response.model_dump_json(indent=2))
    print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
    status = response.status
    print(f'Status: {status}')
    clear_output(wait=True)

print(f'Fine-tuning job {job_id} finished with status: {status}')

# List all fine-tuning jobs for this resource.
print('Checking other fine-tune jobs for this resource.')
response = client.fine_tuning.jobs.list()
print(f'Found {len(response.data)} fine-tune jobs.')

Fine-tuning job ftjob-4c94f760d27141c587b7ac90d74ac538 finished with status: succeeded
Checking other fine-tune jobs for this resource.
Found 1 fine-tune jobs.


## Deploy Fine Tuned Model
Unlike the previous Python SDK commands in this tutorial, since the introduction of the quota feature, model deployment must be done using the REST API, which requires separate authorization, a different API path, and a different API version.

In [15]:
import json
import requests
from dotenv import load_dotenv
load_dotenv()

token= os.getenv("TEMP_AUTH_TOKEN") 
subscription = "4bfc69bf-a717-4a81-aa2b-419e924f242c"  
resource_group = "rg-finetuning"
resource_name = "zzfinetune"
model_deployment_name ="gpt-35-turbo-0613-finetune"

deploy_params = {'api-version': "2023-05-01"} 
deploy_headers = {'Authorization': 'Bearer {}'.format(token), 'Content-Type': 'application/json'}

deploy_data = {
    "sku": {"name": "standard", "capacity": 1}, 
    "properties": {
        "model": {
            "format": "OpenAI",
            "name": "gpt-35-turbo-0613.ft-4c94f760d27141c587b7ac90d74ac538", #retrieve this value from the previous call, it will look like gpt-35-turbo-0613.ft-b044a9d3cf9c4228b5d393567f693b83
            "version": "1"
        }
    }
}
deploy_data = json.dumps(deploy_data)

request_url = f'https://management.azure.com/subscriptions/{subscription}/resourceGroups/{resource_group}/providers/Microsoft.CognitiveServices/accounts/{resource_name}/deployments/{model_deployment_name}'

print('Creating a new deployment...')
r = requests.put(request_url, params=deploy_params, headers=deploy_headers, data=deploy_data)
print(r)
print(r.reason)
print(r.json())

Creating a new deployment...
<Response [201]>
Created
{'id': '/subscriptions/4bfc69bf-a717-4a81-aa2b-419e924f242c/resourceGroups/rg-finetuning/providers/Microsoft.CognitiveServices/accounts/zzfinetune/deployments/gpt-35-turbo-0613-finetune', 'type': 'Microsoft.CognitiveServices/accounts/deployments', 'name': 'gpt-35-turbo-0613-finetune', 'sku': {'name': 'standard', 'capacity': 1}, 'properties': {'model': {'format': 'OpenAI', 'name': 'gpt-35-turbo-0613.ft-4c94f760d27141c587b7ac90d74ac538', 'version': '1'}, 'versionUpgradeOption': 'NoAutoUpgrade', 'capabilities': {'chatCompletion': 'true'}, 'provisioningState': 'Creating', 'rateLimits': [{'key': 'request', 'renewalPeriod': 10, 'count': 1}, {'key': 'token', 'renewalPeriod': 60, 'count': 1000}]}, 'systemData': {'createdBy': 'ziggyzulueta@yeaps.com', 'createdByType': 'User', 'createdAt': '2024-03-28T16:08:33.6126914Z', 'lastModifiedBy': 'ziggyzulueta@yeaps.com', 'lastModifiedByType': 'User', 'lastModifiedAt': '2024-03-28T16:08:33.6126914Z'}

## Use a deployed customized model
After your fine-tuned model is deployed, you can use it like any other deployed model in either the Chat Playground of Azure OpenAI Studio, or via the chat completion API. For example, you can send a chat completion call to your deployed model, as shown in the following Python example. You can continue to use the same parameters with your customized model, such as temperature and max_tokens, as you can with other deployed models.

In [21]:
import os
from openai import AzureOpenAI

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT_FINETUNE"), 
  api_key=os.getenv("AZURE_OPENAI_KEY_FINETUNE"),  
  api_version="2023-05-15"
)

response = client.chat.completions.create(
    model="gpt-35-turbo-0613-finetune", # model = "Custom deployment name you chose for your fine-tuning model"
    messages=[
        {"role": "system", "content": "Clippy is a factual chatbot that is also sarcastic."},
        {"role": "user", "content": "What's the boiling point of water?"},
        #{"role": "assistant", "content": "Yes, customer managed keys are supported by Azure OpenAI."},
        #{"role": "user", "content": "Do other Azure AI services support this too?"}
    ]
)

print(response.choices[0].message.content)

Oh, just a measly 100 degrees Celsius. No biggie.


: 